In [22]:
from utils import *

In [23]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

In [24]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [26]:
from azureml.core.dataset import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [27]:
from azureml.core import Datastore
from azureml.pipeline.core import PipelineData

datastore = Datastore.get(ws, datastore_name="mldata")
data_train = PipelineData('train', datastore=datastore)
data_test = PipelineData('test', datastore=datastore)

In [28]:
from azureml.pipeline.steps import PythonScriptStep

step_1 = PythonScriptStep(name='Preprocessing',
                          script_name="preprocess_output.py",
                          source_directory="code",
                          arguments=[
                              "--input", data_in,
                              "--out-train", data_train,
                              "--out-test", data_test],
                          inputs=[data_in],
                          outputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)


In [29]:
from azureml.pipeline.steps import PythonScriptStep

step_2 = PythonScriptStep(name='Training',
                          script_name="train.py",
                          source_directory="code",
                          arguments=[
                              "--in-train", data_train,
                              "--in-test", data_test],
                          inputs=[data_train, data_test],
                          runconfig=run_conf,
                          compute_target=aml_cluster)

In [30]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[step_1, step_2])

In [31]:
pipeline.validate()

Step Preprocessing is ready to be created [c233a384]
Step Training is ready to be created [3b351df6]


[]

In [32]:
from azureml.core import Experiment

exp = Experiment(ws, "azureml-pipeline")
run = exp.submit(pipeline)

Created step Preprocessing [c233a384][2b975863-239d-4205-bfca-1c396c00cb8a], (This step will run and generate new outputs)Created step Training [3b351df6][a39277d6-6d26-419a-a324-00ef0b8ab6b0], (This step will run and generate new outputs)

Submitted PipelineRun 0f3a8bdf-6131-49e3-bcf1-7fe09c9cc1b9
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0f3a8bdf-6131-49e3-bcf1-7fe09c9cc1b9?wsid=/subscriptions/21dc412b-d9eb-42e7-8317-55bc8eb10cf5/resourcegroups/packt/workspaces/mldemows&tid=e702ab66-1ac1-42ba-b41b-8195d943a1e0


In [33]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …